In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer

In [2]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

def get_token_length(x):
    tokens = tokenizer.encode(x["text"].strip())
    return {
        "token_length": len(tokens)
    }

get_token_length({"text": "Hello"})

{'token_length': 1}

In [5]:
# dataset_name = "SetFit/emotion"
split = "train"
dataset_name = "yelp_polarity"
dataset = load_dataset(dataset_name, split=split) #.map(get_token_length)

TypeError: can only concatenate str (not "int") to str

In [ ]:
import pandas as pd
df = dataset.to_pandas()
df.head()

NameError: name 'dataset' is not defined

In [61]:
df.token_length.describe()

count    560000.000000
mean        174.207863
std         162.654277
min           1.000000
25%          66.000000
50%         125.000000
75%         227.000000
max        2200.000000
Name: token_length, dtype: float64

In [48]:
df.label.value_counts().sum()

2000

In [42]:
def sample(x):
    if len(x) > 1000:
        return x.sample(n=1000)
    else:
        return x
balanced_emotion = df.groupby("label").apply(sample).reset_index(drop=True)
balanced_emotion.label.value_counts()

0    1000
1    1000
2    1000
3    1000
4    1000
5     572
Name: label, dtype: int64

In [49]:
df.to_json(split + ".json", orient="records", lines=True)